In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
from keras.layers import Dropout, Dense, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3

In [ ]:
batch_size=64
data_dir = "data/animals"

train_datagen = ImageDataGenerator(rescale = 1./255,
        validation_split=0.2,
        rotation_range=35,
        width_shift_range=0.25,
        preprocessing_function=tf.keras.applications.resnet.preprocess_input,
        height_shift_range=0.25,
        shear_range=0.25,
        zoom_range=0.25,
        horizontal_flip=True,
        fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale = 1./255,validation_split=0.2)


train_generator = train_datagen.flow_from_directory(data_dir,
    target_size=(299,299),
    class_mode='categorical',
    batch_size=batch_size,
    subset = "training")
validation_generator = validation_datagen.flow_from_directory(data_dir,
    target_size=(299,299),
    class_mode='categorical',
    batch_size=batch_size,
    subset = "validation")

labels = {v: k for k, v in train_generator.class_indices.items()}

In [ ]:
pre_trained_model = InceptionV3(input_shape=(299, 299, 3), include_top=True, weights='imagenet')

for layer in pre_trained_model.layers:
  layer.trainable = False

last_output = pre_trained_model.get_layer('mixed10').output

In [ ]:
x = layers.Dense(1024, activation='relu')(last_output)
x = layers.Dropout(0.2)(x)  
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(90, activation='softmax')(x)           
model = Model(pre_trained_model.input, x) 

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', 
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

In [ ]:
load_model = True
if load_model:
    model = keras.models.load_model('FinalModel.h5')

In [ ]:
if not load_model:
    history = model.fit(train_generator,
                        validation_data=validation_generator,
                        epochs=10)

In [ ]:
model.evaluate(validation_generator)

In [ ]:
#model.save('FinalModel.h5')

In [ ]:
testing_datagen = ImageDataGenerator(rescale = 1./255,validation_split=0)
testing_data = testing_datagen.flow_from_directory('testing_images',
    target_size=(299,299),
    class_mode=None,
    batch_size=1,
    subset="training")

In [ ]:
np.argmax(model.predict(testing_data))

In [ ]:
labels[np.argmax(model.predict(testing_data))]